In [1]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2
!pip install -q datasets~=3.2.0 pydantic~=2.10.4
!pip install -q seqeval~=1.2.2 evaluate~=0.4.3

# 訓練 PII 偵測模型

在這個筆記本中，我們將展示如何使用 `transformers` 套件訓練 PII (個人識別資訊) 偵測模型。我們將使用 `transformers` 套件中的 [`Trainer`](https://huggingface.co/docs/transformers/main_classes/trainer) 類別來微調一個 Encoder-Only 架構的 BERT 模型。我們將利用到標記分類 ([Token classification](https://huggingface.co/docs/transformers/tasks/token_classification)) 進行下游任務 (downstream task) 的訓練。

標記分類為句子中的單個標記分配標籤。最常見的標記分類任務之一是命名實體識別 (NER)。NER 試圖為句子中的每個實體找到一個標籤，例如人名、地點或組織。

In [ ]:
import pandas as pd
import numpy as np
import evaluate

from transformers import (
  AutoTokenizer,
  AutoModelForTokenClassification,
  DataCollatorForTokenClassification,
  pipeline,
  TrainingArguments,
  Trainer,
)
from datasets import load_dataset, DatasetDict

from pydantic import BaseModel
from pprint import pprint

import ast
import torch

# 檢查是否有 GPU 可以使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else device)

## 下載資料

從 Kaggle 下載 PII External Dataset，並解壓縮到 `sample_data` 資料夾。

In [ ]:
# 從 Kaggle 下載 PII External Dataset
!curl -L -o ./sample_data/pii-external-dataset.zip \
  https://www.kaggle.com/api/v1/datasets/download/alejopaullier/pii-external-dataset

# 解壓縮
!unzip -o -q ./sample_data/pii-external-dataset.zip -d ./sample_data/

In [4]:
# The full `train` split
immutable_dataset = load_dataset('csv', data_files='sample_data/pii_dataset.csv', split='train')

### 資料包含什麼？

In [ ]:
# 顯示原始資料中包含的 features 以及筆數
immutable_dataset

In [ ]:
# 檢視資料集中的第一筆資料
pd.set_option('display.max_colwidth', None)
pd.DataFrame(immutable_dataset[:1])

這個表個結構，包含多項資訊，在這個筆記本中，我們將使用：

* `text`: 一個字串，包含 PII 資訊，將用作輸入。
* `token`: 依據 `text` 切割後的字元及字符，將用作訓練。
* `label`: 對應每個 `token` 的標記，其中包含 PII 資訊的類別，將用作訓練。

### 資料前處理

In [ ]:
# 保留必要 features: 'text', 'tokens', 'labels'
dataset = immutable_dataset.remove_columns([
  'document', 'trailing_whitespace', 'prompt', 'prompt_id', 'name',
  'email', 'phone', 'job', 'address', 'username', 'url', 'hobby', 'len'])

# 將 'tokens' 與 'labels' 從 string 轉換為 list (CSV 檔案讀取後會變成 string)
dataset = dataset.map(lambda x: {'tokens': ast.literal_eval(x['tokens']), 'labels': ast.literal_eval(x['labels'])})

# 確認 tokens 長度與 labels 長度相等，避免有缺失的情況,
dataset = dataset.filter(lambda x: len(x['tokens']) == len(x['labels']))

# 將 tokens 欄位重新命名為 words 避免與後面的 tokens 概念混淆
dataset = dataset.rename_column('tokens', 'words')

# 顯示處理後的資料
dataset

In [ ]:
# Reserve 0.1% of the training set for testing
test_dataset = dataset.train_test_split(
  test_size=0.001, # 0.1% of the data is used for testing
  shuffle=False, # Ensure that train and validation sets are the same across runs
  )
# Split into 80% training and 20% validation sets
train_dataset = test_dataset['train'].train_test_split(
  test_size=0.2, # 20% of the data is used for validation
  shuffle=False, # Ensure that train and test sets are the same across runs
  )
dataset = DatasetDict({
  'train': train_dataset['train'],
  'validation': train_dataset['test'],
  'test': test_dataset['test'],
  })
# 顯示處理後的資料
dataset

In [ ]:
# 顯示前 first_n_data 筆資料
first_n_data = 3
pd.set_option('display.max_colwidth', None)
pd.DataFrame(dataset['train'].select(range(first_n_data)))

In [ ]:
# 並列顯示前 max_display 個 words 與 labels
max_display = 50

def show_nth_data(dataset, nth, max_display):
    words = dataset[nth]['words'][:max_display]
    labels = dataset[nth]['labels'][:max_display]
    line1 = ""
    line2 = ""
    for word, label in zip(words, labels): # 逐一取出 word 與 label
        # 計算 word 與 label 的最大長度, 並將 word 與 label 用空白補齊至相同長度
        max_length = max(len(word), len(label))
        line1 += word + " " * (max_length - len(word) + 1)
        line2 += label + " " * (max_length - len(label) + 1)
    pprint(line1, width=200)
    pprint(line2, width=200)
    print()

def show_data(dataset, first_n_data, max_display):
    for i in range(first_n_data):
        show_nth_data(dataset, i, max_display)

show_data(dataset['test'], first_n_data, max_display)

為方便檢視 `token` 與 `label` 的關係，所以並列顯示。

### 資料中的 BIO 標注

IOB 格式（inside, outside, beginning 的縮寫），也常被稱為 BIO 格式，是計算語言學中用於標記任務（例如命名實體識別 NER，詞性標記 POS）的常見標記格式。

* B - for the first token of a named entity
* I - for tokens inside named entity's
* O - for tokens outside any named entity

In [ ]:
# 顯示 BIO 標注
label_names = set()
for data in dataset['train']:
    # 將 labels 欄位轉換為 set 並更新 label_names
    label_names.update(data['labels'])
# convert set to list and sort label names
label_names = list(label_names)
print('=== 資料集中的標籤 ===')
pprint(label_names, compact=True)
print()
print(f'標籤數量: {len(label_names)}')

方便後續訓練過程，我們將建立兩個對照表，一個是將 BIO 標籤映射到一個整數編碼 (One-hot encoding)，另一個是將整數編碼映射回 BIO 標籤。

In [ ]:
# 整數標籤到 BIO 標籤的映射
id2tag = dict(enumerate(label_names))

print('=== id2tag ===')
pprint(sorted(id2tag.items(), key=lambda x: x[0]), compact=True)

In [ ]:
# BIO 標籤到整數標籤的映射
tag2id = dict((v, k) for k, v in id2tag.items())

print('=== tag2id ===')
pprint(sorted(tag2id.items(), key=lambda x: x[1]), compact=True)

## 訓練參數

當我們在微調或訓練機器學習模型時，有幾個重要的參數需要設定。這些參數會影響模型的訓練效果和性能。以下是這些參數的簡單解釋：

* 批次大小（Batch Size）：每次訓練迭代中使用的樣本數量。假設你有1000個數據點，批次大小為32，這意味著模型每次會使用32個數據點來更新權重。當所有數據點都被使用完一次後，這稱為一個 epoch。較大的批次大小可以加速訓練，但需要更多的內存；較小的批次大小則更穩定，但訓練速度較慢。
* 訓練輪數（Epochs）：完整遍歷訓練數據集的次數。如果你設定 epochs 為10，這意味著模型會完整地看10次訓練數據集。更多的 epochs 可以讓模型學習得更充分，但過多的 epochs 可能會導致過擬合（Overfitting / 模型在訓練數據上表現很好，但在新數據上表現不好）。
* 學習率（Learning Rate）：每次更新模型權重時的步伐大小。學習率決定了模型在每次更新時應該調整多少權重。較高的學習率會使模型快速學習，但可能會跳過最佳解；較低的學習率會使模型穩定學習，但訓練時間較長。學習率需要仔細調整，過高或過低都會影響模型的性能。
* 隨機失活（Dropout）：在訓練過程中隨機忽略一些神經元的比例。Dropout 是一種正則化技術，用於防止過擬合。它通過在每次訓練步驟中隨機忽略一些神經元來強制模型學習更穩健的特徵。適當的 Dropout 可以提高模型的泛化能力，但過高的 Dropout 可能會導致模型欠擬合（Underfitting / 模型在訓練數據和新數據上都表現不好）。
* 權重衰減（Weight Decay）：在每次更新權重時，對權重施加的正則化項。Weight Decay 是另一種正則化技術，用於防止過擬合。它通過在每次更新時對權重施加懲罰，使權重保持較小的值。適當的 Weight Decay 可以提高模型的泛化能力，但過高的 Weight Decay 可能會導致模型欠擬合。

這些參數在模型訓練中扮演著重要角色，影響著模型的學習速度、穩定性和泛化能力。調整這些參數需要根據具體的數據集和任務進行實驗和調整，以找到最佳的組合。

### 訓練設定

In [14]:
# 訓練相關設定
class Config(BaseModel):
  model_name: str = 'dslim/distilbert-NER' # 使用蒸餾模型，降低參數量，加快訓練速度
  saved_model_path: str = 'sample_data/saved_decoder_model' # path to save the trained model
  train_batch_size: int = 4 # size of the input batch in training
  eval_batch_size: int = 4 # size of the input batch in evaluation
  epochs: int = 1 # number of times to iterate over the entire training dataset
  lr: float = 2e-5 # learning rate, controls how fast or slow the model learns
  weight_decay: float = 0.01 # weight decay, helps the model stay simple and avoid overfitting by penalizing large weights.
  tags: list # BIO 標註的標籤列表
  id2tag: dict # 整數標籤到 BIO 標籤的映射
  tag2id: dict # BIO 標籤到整數標籤的映射
  num_tags: int # 標籤數量

config = Config(
  tags=label_names,
  id2tag=id2tag,
  tag2id=tag2id,
  num_tags=len(label_names)
)

## Fine-tuning 前的表現

### 載入預訓練分詞器 (Tokenizer)

In [ ]:
# 透過預訓練模型取得 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
  config.model_name,
)
pprint(tokenizer)

確認為 fast tokenizer，方便後續做資料預處理。

> [Fast tokenizers’ special powers](https://huggingface.co/learn/nlp-course/chapter6/3?fw=pt)

In [ ]:
# 確認 tokenizer 是否為 fast tokenizer
tokenizer.is_fast

### 載入預訓練模型

透過 `AutoModelForTokenClassification` 載入預訓練模型，並確認模型的分類數。

In [17]:
model = AutoModelForTokenClassification.from_pretrained(
  config.model_name,
).to(device)

In [ ]:
pprint(model)

這是一個典型的 Encoder-Only 架構，其中包含一個 BERT 模型和一個線性分類器。BERT 模型用於提取特徵，線性分類器用於將特徵映射到分類標籤。

觀察預訓練模型的分類數為 9。

```json
(classifier): Linear(in_features=768, out_features=9, bias=True)
```

### Fine-tuning 前的表現

In [19]:
# 載入預訓練模型
classifier = pipeline(
  task="token-classification",
  model=model,
  tokenizer=tokenizer,
  device=device,
)

In [ ]:
# 合併顯示預測結果
def show_prediction(text, classifier):
    result = classifier(text) # 預測結果
    line1 = ""
    line2 = ""
    for r in result:
        # 取出預測結果中的 word 與 entity
        word = r['word']
        label = r['entity']
        # 計算 word 與 entity 的最大長度, 並將 word 與 entity 用空白補
        max_length = max(len(word), len(label))
        line1 += word + " " * (max_length - len(word) + 1)
        line2 += label + " " * (max_length - len(label) + 1)
    pprint(line1)
    pprint(line2)
    print()

# 顯示預訓練模型預測結果，僅顯示有被標注的部分
for val in dataset['test']: # 逐一取出測試資料
  print(f'輸入: {val["text"]}')
  show_prediction(val["text"], classifier)

## 訓練模型

### 資料預處理

#### 了解 Tokenizer 行為

分詞器添加了模型使用的特殊標記（[CLS] 在開頭和 [SEP] 在結尾），並且大多數單詞保持不變。然而，某些單詞 (word) 會被分為數個子詞 (subword)。


In [ ]:

# 以第 data_nth 筆資料為例
data_cat = 'test'
data_nth = 2
input_words = dataset[data_cat][data_nth]["words"]
input_labels = dataset[data_cat][data_nth]["labels"]
# 對 input_words 進行分詞
input_token_ids = tokenizer(
  input_words,
  # is_split_into_words: Whether or not the input is already pre-tokenized (e.g., split into words).
  # If set to True, the tokenizer assumes the input is already split into words (for instance, by splitting it on whitespace) which it will tokenize.
  # This is useful for NER or token classification.
  is_split_into_words=True)

# 如我們所見，分詞器添加了模型使用的特殊標記（[CLS] 在開頭和 [SEP] 在結尾），
# 並且大多數單詞保持不變。然而，某些單詞 (word) 會被分為數個子詞 (subword)，如: Bar, ##eil 和 ##ly
pprint(input_token_ids.tokens(), compact=True)

In [ ]:
# 原始資料
print(f'length of input_words: {len(input_words)}')
print(f'length of input_labels: {len(input_labels)}')
# 這導致了我們的輸入和標籤之間的不匹配
print(f'length of token id: {len(input_token_ids.tokens())}')

經過 Tokenizer 後，這導致了我們的輸入和標籤之間的不匹配。

感謝 fast tokenizer 我們可以輕鬆地將每個 Token 藉由 `word_ids()` 映射到其對應的單詞 Word IDs。

舉例：

| Word    | Token | Word ID |
|---------|-------|---------|
|         | [CLS] | None    |
| My      | My    | 0       |
| name    | name  | 1       |
| is      | is    | 2       |
| Ludmila | Lu    | 3       |
|         | ##d   | 3       |
|         | ##mi  | 3       |
|         | ##la  | 3       |
|         | [SEP] | None    |


In [ ]:
# 資料對比
print('=== Tokenizer 前 ===')
show_nth_data(dataset[data_cat], data_nth, max_display)
print('=== Tokenizer 後 ===')
pprint(input_token_ids.tokens()[:max_display], compact=True)

# `word_ids` return the list of tokens (sub-parts of the input strings after word/subword splitting and before conversion to integer indices)
# at a given batch index (only works for the output of a fast tokenizer).
print()
print('=== 對應的 word_ids ===')
pprint(input_token_ids.word_ids()[:max_display], compact=True)


#### 重新校準 Tokenizer 與標籤

我們可以擴展標籤以匹配單詞 Token。

1. 首先，我們將應用的規則是特殊 Token 獲得 -100 標籤。這是因為默認情況下，-100 是我們在損失函數中被忽略的索引。
2. 然後，每個 Token 獲得與其所在單詞相同的標籤，因為它們是同一實體的一部分。

In [24]:
def align_labels_with_tokens(word_ids, labels):
    new_labels = []
    current_word_id = None
    for word_id in word_ids:
        if word_id is None: # None 代表特殊 Token
            label = -100
        elif word_id != current_word_id: # Word ID 改變，代表新的 Token
            current_word_id = word_id # 更新 Word ID
            label = -100 if word_id is None else config.tag2id.get(labels[word_id]) # 取得對應的標籤
        else: # 與前一個 Token 相同的字
            label = config.tag2id.get(labels[word_id])
        new_labels.append(label) # 附加新的標籤
    # 返回新的標籤
    return new_labels

In [ ]:
labels = align_labels_with_tokens(input_token_ids.word_ids(), input_labels)

# 顯示對齊後的標籤，兩者數量應該相同
print(f'length of token id: {len(input_token_ids.tokens())}')
print(f'length of labels: {len(labels)}')

經過重新校準後，我們的標籤與 Token 一一對應。

| Word    | Token | Word ID | Label          |
|---------|-------|---------|----------------|
|         | [CLS] | None    | -              |
| My      | My    | 0       | O              |
| name    | name  | 1       | O              |
| is      | is    | 2       | O              |
| Ludmila | Lu    | 3       | B-NAME_STUDENT |
|         | ##d   | 3       | B-NAME_STUDENT |
|         | ##mi  | 3       | B-NAME_STUDENT |
|         | ##la  | 3       | B-NAME_STUDENT |
|         | [SEP] | None    | -              |


In [ ]:
print('=== 對應的標籤 ===')
pprint(input_token_ids.tokens()[:max_display], compact=True)
# 將整數標籤轉換為 BIO 標籤方便比較
labels_id = [config.id2tag.get(label, '-') for label in labels]
pprint(labels_id[:max_display], compact=True)


#### 定義預處理函數

要預處理整個數據集，我們需要對所有輸入進行分詞，並對所有標籤應用 `align_labels_with_tokens()`。為了利用快速分詞器的速度，最好一次分詞大量文本，因此我們將編寫一個批次處理函數。

與之前的示例不同的是，當分詞器的輸入是文本列表時，`word_ids()` 函數需要獲取我們想要單詞 ID 的索引。

In [27]:
def tokenize_and_align_labels(dataset):
    # 使用 tokenizer 對資料集進行分詞
    tokenized_inputs = tokenizer(
        dataset['words'], truncation=True, is_split_into_words=True
    )
    all_labels = dataset['labels']
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i) # 取得對應的 word_ids
        new_labels.append(align_labels_with_tokens(word_ids, labels)) # 對齊標籤
    # 更新 labels 欄位，請留意這邊的 labels 欄位是整數標籤，而非 BIO 標籤
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = tokenize_and_align_labels(dataset[data_cat][:1])
input_ids = tokenized_dataset['input_ids'][0]
labels = tokenized_dataset['labels'][0]
line1 = ""
line2 = ""
for input_id, label in zip(input_ids, labels): # 逐一取出 input_id 與 label
    # 計算 input_id 與 label 的最大長度, 並將 input_id 與 label 用空白補齊至相同長度
    str_input_id = str(input_id)
    str_label = str(label)
    max_length = max(len(str_input_id), len(str_label))
    line1 += str_input_id + " " * (max_length - len(str_input_id) + 1)
    line2 += str_label + " " * (max_length - len(str_label) + 1)
pprint(line1, width=200)
pprint(line2, width=200)

請留意經過批次轉換後，labels 欄位是整數標籤，而不是 BIO 字串標籤。

#### 批次校準 Tokenizer 與標籤

使用 `Dataset.map()` 方法，選項設置為 `batched=True`。

In [ ]:
tokenized_dataset = dataset.map(
    tokenize_and_align_labels, # 對資料集進行分詞與標籤對齊
    batched=True, # 是否以批次進行處理
    remove_columns=dataset['train'].column_names, # 移除不必要的欄位
)

In [ ]:
# 顯示前 first_n_data 筆資料
pd.DataFrame(tokenized_dataset[data_cat].select(range(first_n_data)))

### 資料校對器 (Data Collator)

我們不能僅使用 `DataCollatorWithPadding`，因為它只填充輸入（input IDs, attention mask）。在這裡，我們的標籤應該以與輸入完全相同的方式進行填充，以保持相同的大小，使用 -100 作為值，以便在損失計算中忽略相應的預測。

這一切都由 `DataCollatorForTokenClassification` 完成。與 `DataCollatorWithPadding` 一樣，它需要使用預處理輸入的分詞器：

>
> * [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorWithPadding) that will dynamically pad the inputs received.
> * [DataCollatorForTokenClassification](https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorForTokenClassification) that will dynamically pad the inputs received, as well as the labels.
>

In [31]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
# 展示 DataCollatorForTokenClassification 的輸出, 標籤以 -100 表示 padding
batch = data_collator([tokenized_dataset[data_cat][i] for i in range(first_n_data)])
pprint(batch["labels"])

### 模型評估函數

在訓練過程中包含度量標準通常有助於評估模型的性能。您可以使用 Evaluate 庫快速加載評估方法。對於這個任務，請加載 [seqeval](https://huggingface.co/docs/evaluate/a_quick_tour) 框架。Seqeval 實際上會生成多個分數：precision, recall, F1, 和 accuracy。

* Precision: 精確率，是指所有被標記為正的樣本中實際為正的比例。

$\ Precision = \frac{\text{correctly classified actual positives}}{\text{everything classified as positives}} = \frac{TP}{TP + FP} $

* Recall: 召回率，是指所有實際為正的樣本中被標記為正的比例。

$\ Recall = \frac{\text{correctly classified actual positives}}{\text{all actual positives}} = \frac{TP}{TP + FN} $

* F1: F1 值是精確率和召回率的調和平均值，用於綜合考慮精確率和召回率。

$\ F1 = 2 \times \frac{Precision \times Recall}{Precision + Recall} $

* Accuracy: 準確率，是指所有被正確分類的樣本數量與總樣本數量之比。

$\ Accuracy = \frac{\text{correctly classifications}}{\text{total classifications}} = \frac{TP + TN}{TP + TN + FP + FN} $


In [33]:
seqeval = evaluate.load("seqeval")

def compute_metrics(eval_preds):
    # Unpack logits and labels from the input
    logits, labels = eval_preds

    # Convert logits to the index of the maximum logit value
    predictions = np.argmax(logits, axis=-1)

    # Map predictions and labels to their corresponding label names, ignoring padding (-100)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Compute evaluation metrics using seqeval
    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    # Return the computed metrics
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

### 載入預訓練模型

您現在可以開始訓練您的模型了！使用 `AutoModelForTokenClassification` 加載預訓練的模型，並指定預期標籤的數量和標籤映射：

需要留意，因為預訓練模型的分類數為 9，所以我們需要重新設定模型的分類數，且忽略預訓練模型的分類層。

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
  config.model_name,
  num_labels=config.num_tags, # 任務標籤數量
  ignore_mismatched_sizes=True, # 忽略不匹配的大小，預訓練模型的標籤數量與我們的標籤數量不同
  id2label=config.id2tag, # 整數標籤到 BIO 標籤的映射
  label2id=config.tag2id, # BIO 標籤到整數標籤的映射
  )

In [ ]:
# 查看可訓練的參數量約 65M
print('Parameters: {:,}, Trainable Parameters: {:,}'.format(
  model.num_parameters(),
  model.num_parameters(only_trainable=True)))

### 定義訓練參數

In [ ]:
training_args = TrainingArguments(
  output_dir='sample_data/train_output_pii_detection',
  learning_rate=config.lr,
  per_device_train_batch_size=config.train_batch_size,
  per_device_eval_batch_size=config.eval_batch_size,
  num_train_epochs=config.epochs,
  weight_decay=config.weight_decay,
  eval_strategy='epoch', # 每個 epoch 評估一次
  save_strategy='epoch', # 每個 epoch 儲存一次
  load_best_model_at_end=True,
  report_to='none', # Disable wandb on colab
  save_total_limit=5, # 最多儲存 5 個 checkpoints
)

trainer = Trainer(
  model=model,
  tokenizer=tokenizer,
  args=training_args,
  train_dataset=tokenized_dataset['train'],
  eval_dataset=tokenized_dataset['validation'],
  data_collator=data_collator,
  compute_metrics=compute_metrics,
)

### 開始訓練

In [ ]:
# 開始訓練，這可能需要一些時間
trainer.train()

訓練完成後，您可以通過運行 `Trainer.evaluate()` 方法在驗證集上評估模型的性能。它會計算模型的損失和其他評估指標，並返回這些結果。這對於了解模型在未見數據上的表現非常有用。

In [ ]:
trainer.evaluate()

#### 保存模型

In [38]:
# 儲存模型
trainer.save_model(config.saved_model_path)

## 評估模型

### 載入微調後 Tokenizer

In [39]:
tokenizer = AutoTokenizer.from_pretrained(
  config.saved_model_path,
)

### 載入微調後模型

請留意分類數已經從預訓練模型的 9 類，變成 10 類。

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
  config.saved_model_path,
).to(device)
pprint(model)

### Fine-tuning 後的表現

In [ ]:
# 載入新模型
classifier = pipeline(
  task="token-classification",
  model=config.saved_model_path,
  device=device,)

# 顯示預訓練模型預測結果，僅顯示有被標注的部分
for val in dataset['test']: # 逐一取出測試資料
  print(f'輸入: {val["text"]}')
  show_prediction(val["text"], classifier)